<a href="https://colab.research.google.com/github/s4d3/NMT-Implementation/blob/main/OpenNMT_2_JaId.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# OpenNMT-py によるニューラル機械翻訳演習
# Neural machine translation exercise with OpenNMT-py
OpenNMT-py というツールを用いて英日機械翻訳の演習を行います。
Google Colaboratory は GPU を用いた実験ができるので、GPU を用いてニューラル機械翻訳の実験をしましょう。（「編集」→「ノートブックの設定」から「ハードウェアアクセラレータ」を GPU にしておいてください。以下、Python 3 で説明します。）
データは田中コーパスという自由に使える対訳コーパスを使用します。翻訳結果を実際に見て評価する、ということも演習の目的です。

We will practice English-Japanese machine translation using a tool called OpenNMT-py.
Google Colaboratory can experiment with GPU, so let's experiment with neural machine translation using GPU. (Please set "Hardware Accelerator" to GPU from "Edit"-> "Notebook Settings". It will be explained in Python 3 below.)
The data uses a freely available bilingual corpus called Tanaka Corpus. The purpose of the exercise is to actually see and evaluate the translation results.

# OpenNMT-py のインストール
# Install OpenNMT-py

## pip によるインストール
## Installation with pip
OpenNMT-py を使って実験を行います。かなり開発が速く、数週間で内容が変わってしまうので、ここでは2020年8月にリリースされた v1.2.0 というバージョンを使います。pip というのは Python のソフトウェアをインストールするときに使うツールの名前で、依存関係も含めて自動でセットアップしてくれます。

Experiment with OpenNMT-py. It's pretty fast to develop and changes in a few weeks, so I'm going to use the version v1.2.0 released in August 2020. pip is the name of the tool you use to install Python software, and it will set it up automatically, including its dependencies.

In [ ]:
#!pip install OpenNMT-py==1.2.0
!pip install --upgrade OpenNMT-py==2.0.0rc1

# 対訳データのインストールと前処理
# Bilingual data installation and preprocessing

## 田中コーパス
## Tanaka Corpus

田中コーパスというデータで翻訳実験を行います。日本の大学生が翻訳したデータです。そのうち適当な長さの5万文だけを抽出して実験しやすいようにしてくれたデータを用います。
train.{ja,en}というのが学習データ、dev.{ja,en}というのが開発データ、test.{ja,en}というのがテストデータです。

We will conduct a translation experiment using data called Tanaka Corpus. Data translated by Japanese university students. We will use the data that makes it easier to experiment by extracting only 50,000 sentences of appropriate length.
train. {ja, en} is the training data, dev. {ja, en} is the development data, and test. {Ja, en} is the test data.

In [3]:
#ls small_parallel_enja
!ls /content/drive/MyDrive/OpenNMT/dataJaId06

dev.id	dev.ja	ja-id.yaml  run  test.id  test.ja  train.id  train.ja


## データの前処理
## Data preprocessing

onmt_preprocess というスクリプトでデータの前処理をします。語彙のサイズをコントロールしたり、単語分割をしたりします。

Preprocess the data with a script called onmt_preprocess. Control the size of the vocabulary and split words.

In [ ]:
!onmt_train -config /content/drive/MyDrive/OpenNMT/dataJaId06/ja-id.yaml

In [ ]:
#onmt_translate -model toy-ende/run/model_step_1000.pt -src toy-ende/src-test.txt -output toy-ende/pred_1000.txt -gpu 0 -verbose
!onmt_translate -model /content/drive/MyDrive/OpenNMT/dataJaId06/run/modelALT_step_20000.pt -src /content/drive/MyDrive/OpenNMT/dataJaId06/test.ja -output /content/drive/MyDrive/OpenNMT/dataJaId06/pred_1000.txt -gpu 0 -verbose

# NMTの訓練
# NMT training

## 学習
## Learning

onmt_train というスクリプトで学習します。いくつかオプションがあるので、変えてみると挙動も変わります。
--world_size で使う GPU の個数、--gpu_ranks で使う GPU を指定します（0スタートです）。指定しないと CPU になってしまいます。CPU は GPU の数十倍遅いです。
--train_steps は学習するステップを指定します。40,000だとこのデータは20分弱で学習できます。

You will learn with a script called onmt_train. There are several options, so if you change it, the behavior will change.
Specify the number of GPUs to use with --world_size and the GPUs to use with --gpu_ranks (starting from 0). If not specified, it will be the CPU. The CPU is dozens of times slower than the GPU.
--train_steps specifies the steps to learn. At 40,000, this data can be learned in less than 20 minutes.

## 翻訳モデル
## Translation model

学習されたモデルは _step_10000.pt のような名前で保存されています。
The trained model is saved with a name like _step_10000.pt.

# NMTによる翻訳
Translated by NMT

## デコード
## Decode

onmt_translate というスクリプトでデコードします。一つ前に学習した翻訳モデルのうちの1つを指定します。
--output pred.txt というので結果を保存するファイルを指定します。
--replace_unk というのは未知語処理をするというオプションです。
--verbose は結果を冗長に画面にも表示させるオプションです。

Decode with a script called onmt_translate. Specify one of the previously learned translation models.
Since it is --output pred.txt, specify the file to save the result.
--replace_unk is an option to process unknown words.
--verbose is an option to display the result redundantly on the screen.

In [ ]:
!cat /content/drive/MyDrive/OpenNMT/dataJaId06/pred_1000.txt

##BLEU による評価
##Evaluation by BLEU

正解（参照訳）のファイルを第一引数に、システムの出力をリダイレクト（<）で渡します。
sacrebleu というツールで評価するので、pip でインストールします。

Pass the system output by redirect (<) with the correct answer (reference translation) file as the first argument.
Since it is evaluated by a tool called sacrebleu, install it with pip.

In [ ]:
!pip install sacrebleu

In [12]:
!chmod 755 -R /content/drive/MyDrive/Marian-NMT/marian/examples/tools/

In [ ]:
!/content/drive/MyDrive/Marian-NMT/marian/examples/tools/moses-scripts/scripts/tokenizer/detokenizer.perl < /content/drive/MyDrive/OpenNMT/dataJaId06/pred_1000.txt > /content/drive/MyDrive/OpenNMT/dataJaId06/predTok_1000.txt
!/content/drive/MyDrive/Marian-NMT/marian/examples/tools/moses-scripts/scripts/tokenizer/detokenizer.perl < /content/drive/MyDrive/OpenNMT/dataJaId06/test.id > /content/drive/MyDrive/OpenNMT/dataJaId06/testTok.id


In [15]:
!sacrebleu /content/drive/MyDrive/OpenNMT/dataJaId06/testTok.id < /content/drive/MyDrive/OpenNMT/dataJaId06/predTok_1000.txt


BLEU+case.mixed+numrefs.1+smooth.exp+tok.13a+version.1.4.14 = 4.0 33.8/7.7/2.2/0.7 (BP = 0.883 ratio = 0.889 hyp_len = 23138 ref_len = 26013)


## 結果の保存
## Save results

画面に表示して保存する場合は ``cat`` というコマンドを使って表示させます。複数の結果を表示させたい場合は ``paste`` というコマンドを使います。
結果をファイルに保存する場合は ``files.download`` というメソッドを呼びます。
出力されていない結果は24時間で消えるので、ご注意ください。

To display it on the screen and save it, use the command `` cat`` to display it. If you want to display multiple results, use the command `` paste``.
If you want to save the result to a file, call the method `` files.download``.
Please note that unoutput results will disappear in 24 hours.